In [4]:
import pandas as pd 
import numpy as np

file1 = 'c:/users/wston/documents/python/pybuild/TSLA_x.csv'

jump = pd.read_csv(file1)
datetime_series = pd.DatetimeIndex(jump['Date'])
datetime_series
jump = jump.set_index(datetime_series)
jump
# jump.to_csv('c:/users/wston/documents/stock_data.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2010-06-30,2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2010-07-01,2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
2010-07-02,2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
2010-07-06,2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500
2010-07-07,2010-07-07,1.093333,1.108667,0.998667,1.053333,1.053333,103825500
...,...,...,...,...,...,...,...
2019-12-13,2019-12-13,24.070000,24.347334,23.642668,23.892668,23.892668,98563500
2019-12-16,2019-12-16,24.170000,25.573999,24.166668,25.433332,25.433332,272613000
2019-12-17,2019-12-17,25.266001,25.700001,25.059999,25.266001,25.266001,127452000


In [1]:

num_obs = jump[['Close'
]].count()
num_obs


def add_memory(s,n_days=50,mem_strength=0.1):
    ''' adds autoregressive behavior to series of data'''
    add_ewm = lambda x: (1-mem_strength)*x + mem_strength*x.ewm(n_days).mean()
    out = s.groupby(level='Date').apply(add_ewm)
    return out

# generate feature data
f01 = pd.Series(np.random.randn(2386),index=jump[['Close']].index)
f01 = add_memory(f01,10,0.1)
f02 = pd.Series(np.random.randn(2386),index=jump[['Close']].index)
f02 = add_memory(f02,10,0.1)
f03 = pd.Series(np.random.randn(2386),index=jump[['Close']].index)
f03 = add_memory(f03,10,0.1)
f04 = pd.Series(np.random.randn(2386),index=jump[['Close']].index)
f04 = f04 # no memory

features = pd.concat([f01,f02,f03,f04],axis=1)
features.to_csv('c:/users/wston/documents/jump_data.csv')

outcome =   f01 * np.linspace(0.5,1.5,2386) + \
            f02 * np.linspace(1.5,0.5,2386) + \
            f03 * pd.Series(np.sin(2*np.pi*np.linspace(0,1,2386)*2)+1,index=f03.index) + \
            f04 + \
            np.random.randn(2386) * 3 
outcome.name = 'outcome'
outcome

NameError: name 'jump' is not defined

In [6]:
from sklearn.linear_model import LinearRegression
from itertools import chain

recalc_dates = features.resample('D',level='Date').mean().values[:-1]
# print('init', str(recalc_dates))

flat = list(chain.from_iterable(recalc_dates))
print(flat)
recalc_dates = pd.Series(flat)
models = pd.Series(index=recalc_dates)
recalc_dates = pd.series(dtype='float64')
for date in recalc_dates:
    X_train = features.iloc[0:2386]
    y_train = outcome.iloc[0:2386]
    model = LinearRegression()
    model.fit(X_train,y_train)
    models.loc[date] = model

begin_dates = models.index
end_dates = models.index[1:].append(pd.to_datetime(['2099-12-31']))

predictions = pd.Series(index=features.index)

for i,model in enumerate(models): #loop thru each models object in collection
    X = features.iloc[0:2386]
    p = pd.Series(model.predict(X),index=X.index)
    predictions.loc[X.index] = p

[0.20315593593486223, -0.7804002590690181, 1.0855791605577527, -1.2394178174608383, -1.3468723729032508, 0.4640636217917525, 0.9829061670388921, 0.8393352570662144, -0.039501677594787246, 0.02985584237564583, 0.5547658365996543, 0.623531383659878, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, -0.8777128861981508, 0.3622246131845417, -1.2714264615689448, -0.533666023495633, 0.36258696656915274, -1.6226853201520803, 0.7761245140777807, -0.3136147406331749, -0.6385453438039284, 1.098996387512931, -0.10329545430735147, 0.4669878453895261, -0.12146860027330324, -0.4944743220634181, -2.352644506492158, -0.48791252332258306, nan, nan, nan, nan, nan, nan, nan, nan, -0.7830562815977757, -1.3707627728975338, -0.03344231902785254, -0.003127014483351594, -1.3922605264411965, 0.22545743091783252, 0.0021606933924887303, 1.4923940173017753, 1.4420086907304734, -0.21039721826214489, -2.1992052005637257, -0.3566470457425239, 2.2554122230842593, -1.4419381314399313, -0.5144005867385816, -1

C:\Users\wston\AppData\Local\Temp/ipykernel_14004/2903425554.py:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  models = pd.Series(index=recalc_dates)


AttributeError: module 'pandas' has no attribute 'series'

In [ ]:
from sklearn.linear_model import LinearRegression
from itertools import chain

recalc_dates = features.resample('D',level='Date').mean().values[:-1]
# print('init', str(recalc_dates))

flat = list(chain.from_iterable(recalc_dates))
print(flat)
recalc_dates = pd.Series(flat)
models = pd.Series(index=recalc_dates)
recalc_dates = pd.series(dtype='float64')
for date in recalc_dates:
    X_train = features.iloc[0:2386]
    y_train = outcome.iloc[0:2386]
    model = LinearRegression()
    model.fit(X_train,y_train)
    models.loc[date] = model

begin_dates = models.index
end_dates = models.index[1:].append(pd.to_datetime(['2099-12-31']))

predictions = pd.Series(index=features.index)

for i,model in enumerate(models): #loop thru each models object in collection
    X = features.iloc[0:2386]
    p = pd.Series(model.predict(X),index=X.index)
    predictions.loc[X.index] = p

[0.20315593593486223, -0.7804002590690181, 1.0855791605577527, -1.2394178174608383, -1.3468723729032508, 0.4640636217917525, 0.9829061670388921, 0.8393352570662144, -0.039501677594787246, 0.02985584237564583, 0.5547658365996543, 0.623531383659878, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, -0.8777128861981508, 0.3622246131845417, -1.2714264615689448, -0.533666023495633, 0.36258696656915274, -1.6226853201520803, 0.7761245140777807, -0.3136147406331749, -0.6385453438039284, 1.098996387512931, -0.10329545430735147, 0.4669878453895261, -0.12146860027330324, -0.4944743220634181, -2.352644506492158, -0.48791252332258306, nan, nan, nan, nan, nan, nan, nan, nan, -0.7830562815977757, -1.3707627728975338, -0.03344231902785254, -0.003127014483351594, -1.3922605264411965, 0.22545743091783252, 0.0021606933924887303, 1.4923940173017753, 1.4420086907304734, -0.21039721826214489, -2.1992052005637257, -0.3566470457425239, 2.2554122230842593, -1.4419381314399313, -0.5144005867385816, -1

C:\Users\wston\AppData\Local\Temp/ipykernel_14004/2903425554.py:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  models = pd.Series(index=recalc_dates)


AttributeError: module 'pandas' has no attribute 'series'

In [ ]:
import sklearn.metrics as metrics

# make sure we have 1-for-1 mapping between pred and true
common_idx = outcome.dropna().index.intersection(predictions.dropna().index)
y_true = outcome[common_idx]
y_true.name = 'y_true'
y_pred = predictions[common_idx]
y_pred.name = 'y_pred'

standard_metrics = pd.Series()

standard_metrics.loc['explained variance'] = metrics.explained_variance_score(y_true, y_pred)
standard_metrics.loc['MAE'] = metrics.mean_absolute_error(y_true, y_pred)
standard_metrics.loc['MSE'] = metrics.mean_squared_error(y_true, y_pred)
standard_metrics.loc['MedAE'] = metrics.median_absolute_error(y_true, y_pred)
standard_metrics.loc['RSQ'] = metrics.r2_score(y_true, y_pred)

print(standard_metrics)

In [ ]:
print(pd.concat([y_pred,y_true],axis=1).tail())

In [ ]:
def make_df(y_pred,y_true):
    y_pred.name = 'y_pred'
    y_true.name = 'y_true'
    
    df = pd.concat([y_pred,y_true],axis=1)

    df['sign_pred'] = df.y_pred.apply(np.sign)
    df['sign_true'] = df.y_true.apply(np.sign)
    df['is_correct'] = 0
    df.loc[df.sign_pred * df.sign_true > 0 ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
    df['is_incorrect'] = 0
    df.loc[df.sign_pred * df.sign_true < 0,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
    df['is_predicted'] = df.is_correct + df.is_incorrect
    df['result'] = df.sign_pred * df.y_true 
    return df

df = make_df(y_pred,y_true)
# print(df.dropna().tail())
df

In [ ]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()
    
    return scorecard    

calc_scorecard(df)

In [ ]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    return scorecard    

calc_scorecard(df)

In [ ]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    # metrics for a subset of predictions
    scorecard.loc['edge_long'] = df[df.sign_pred == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_short'] = df[df.sign_pred == -1].result.mean()  - df.y_true.mean()

    scorecard.loc['edge_win'] = df[df.is_correct == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_lose'] = df[df.is_incorrect == 1].result.mean()  - df.y_true.mean()

    return scorecard    

calc_scorecard(df)

In [ ]:
def scorecard_by_year(df):
    df['year'] = df.index.get_level_values('Date').year
    return df.groupby('year').apply(calc_scorecard).T

print(scorecard_by_year(df))

In [ ]:
def scorecard_by_symbol(df):
    return df.groupby(level='Date').apply(calc_scorecard).T

print(scorecard_by_symbol(df))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor

X_train,X_test,y_train,y_test = train_test_split(features,outcome,test_size=0.20,shuffle=False)

# linear regression
model1 = LinearRegression().fit(X_train,y_train)
model1_train = pd.Series(model1.predict(X_train),index=X_train.index)
model1_test = pd.Series(model1.predict(X_test),index=X_test.index)

model2 = RandomForestRegressor().fit(X_train,y_train)
model2_train = pd.Series(model2.predict(X_train),index=X_train.index)
model2_test = pd.Series(model2.predict(X_test),index=X_test.index)

# create dataframes for each 
model1_train_df = make_df(model1_train,y_train)
model1_test_df = make_df(model1_test,y_test)
model2_train_df = make_df(model2_train,y_train)
model2_test_df = make_df(model2_test,y_test)

s1 = calc_scorecard(model1_train_df)
s1.name = 'model1_train'
s2 = calc_scorecard(model1_test_df)
s2.name = 'model1_test'
s3 = calc_scorecard(model2_train_df)
s3.name = 'model2_train'
s4 = calc_scorecard(model2_test_df)
s4.name = 'model2_test'

print(pd.concat([s1,s2,s3,s4],axis=1))